In [1]:
import time
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display

# ------------------ Naive Search ------------------
def naive_search(pat, txt):
    n, m = len(txt), len(pat)
    occurrences = []
    for i in range(n - m + 1):
        match = True
        for j in range(m):
            if txt[i + j] != pat[j]:
                match = False
                break
        if match:
            occurrences.append(i)
    print(f"Naive: Pattern '{pat}' in text -> {occurrences if occurrences else 'None'}")
    return occurrences

# ------------------ KMP Search ------------------
def compute_lps(pattern):
    m = len(pattern)
    lps = [0] * m
    length = 0
    i = 1
    while i < m:
        if pattern[i] == pattern[length]:
            length += 1
            lps[i] = length
            i += 1
        else:
            if length != 0:
                length = lps[length - 1]
            else:
                lps[i] = 0
                i += 1
    return lps

def kmp_search(pat, txt, animate=True, interval=250):
    n, m = len(txt), len(pat)
    lps = compute_lps(pat)
    occurrences = []
    step_data = []
    i = 0  # index for txt
    j = 0  # index for pat
    while i < n:
        step_data.append((i, j, occurrences.copy(), "Compare"))
        if pat[j] == txt[i]:
            i += 1
            j += 1
            if j == m:
                occurrences.append(i - j)
                step_data.append((i, j, occurrences.copy(), "Match Found"))
                j = lps[j - 1]
        else:
            if j != 0:
                j = lps[j - 1]
                step_data.append((i, j, occurrences.copy(), "Fallback"))
            else:
                i += 1
                step_data.append((i, j, occurrences.copy(), "Advance"))

    print(f"KMP: Pattern '{pat}' in text -> {occurrences if occurrences else 'None'}")

    if not animate:
        return occurrences

    # Visualization
    fig, ax = plt.subplots(figsize=(max(6, n * 0.25), 2.5))
    ax.set_xlim(-0.5, n - 0.5)
    ax.set_ylim(-2, 3)
    ax.axis("off")

    # Text display
    for idx, ch in enumerate(txt):
        rect = plt.Rectangle((idx - 0.4, 0.5), 0.8, 0.8, fill=False)
        ax.add_patch(rect)
        ax.text(idx, 0.9, ch, ha="center", va="center", fontsize=12)

    # Pattern boxes (will shift)
    pattern_boxes = [plt.Rectangle((i - 0.4, -0.5), 0.8, 0.8, fill=False) for i in range(m)]
    for rect in pattern_boxes:
        ax.add_patch(rect)
    pattern_texts = [ax.text(i, -0.1, pat[i], ha="center", va="center", fontsize=12) for i in range(m)]

    status = ax.text(0, 2.0, "", ha="left", va="center", fontsize=9)
    lps_display = ax.text(0, 2.4, f"LPS: {lps}", ha="left", va="center", fontsize=8)

    def update(frame):
        i, j, occ, action = frame
        alignment_shift = i - j
        # reset and align pattern
        for idx in range(m):
            pattern_boxes[idx].set_xy((idx + alignment_shift - 0.4, -0.5))
            pattern_texts[idx].set_x(idx + alignment_shift)
            pattern_boxes[idx].set_edgecolor("black")

        # clear prior overlays
        for patch in [p for p in ax.patches if getattr(p, "custom_occ", False)]:
            patch.remove()

        # highlight found occurrences
        for oc in occ:
            for k in range(m):
                overlay = plt.Rectangle((oc + k - 0.4, 0.5), 0.8, 0.8, fill=True, alpha=0.25)
                overlay.custom_occ = True
                ax.add_patch(overlay)

        # highlight current comparison
        if 0 <= i - 1 < len(txt) and 0 <= j - 1 < m:
            pattern_boxes[j - 1].set_edgecolor("red")
            rect = plt.Rectangle((i - 1 - 0.4, 0.5), 0.8, 0.8, fill=False, edgecolor="red", linewidth=2)
            rect.custom_occ = True
            ax.add_patch(rect)

        status.set_text(f"KMP: i={i}, j={j}, Action={action}, Matches={occ}")
        return pattern_boxes + pattern_texts + [status, lps_display]

    anim = FuncAnimation(fig, update, frames=step_data, interval=interval, blit=False, repeat=False)
    plt.close(fig)
    display(HTML(anim.to_jshtml()))
    return occurrences

# ------------------ Rabin-Karp Search ------------------
def rabin_karp_search(pat, txt, base=256, mod=101, animate=True, interval=300):
    n, m = len(txt), len(pat)
    if m > n:
        print(f"Rabin-Karp: Pattern '{pat}' longer than text; no match.")
        return []

    # Precompute high-order multiplier
    h = pow(base, m - 1, mod)
    pat_hash = 0
    window_hash = 0
    occurrences = []
    step_data = []

    # Initial hash computation
    for i in range(m):
        pat_hash = (base * pat_hash + ord(pat[i])) % mod
        window_hash = (base * window_hash + ord(txt[i])) % mod
    step_data.append((0, window_hash, pat_hash, occurrences.copy(), "Initial"))

    # Slide window
    for i in range(n - m + 1):
        if i > 0:
            # Rolling hash update
            window_hash = (window_hash - ord(txt[i - 1]) * h) % mod
            window_hash = (window_hash * base + ord(txt[i + m - 1])) % mod
            window_hash = (window_hash + mod) % mod  # keep positive
            step_data.append((i, window_hash, pat_hash, occurrences.copy(), "Roll"))

        if window_hash == pat_hash:
            # Check actual substring to avoid spurious hit
            if txt[i:i + m] == pat:
                occurrences.append(i)
                step_data.append((i, window_hash, pat_hash, occurrences.copy(), "Match Verified"))
            else:
                step_data.append((i, window_hash, pat_hash, occurrences.copy(), "Spurious Hit"))
        else:
            step_data.append((i, window_hash, pat_hash, occurrences.copy(), "No Match"))

    print(f"Rabin-Karp: Pattern '{pat}' in text -> {occurrences if occurrences else 'None'} (mod={mod})")

    if not animate:
        return occurrences

    # Visualization
    fig, ax = plt.subplots(figsize=(max(6, n * 0.25), 3.5))
    ax.set_xlim(-0.5, n - 0.5)
    ax.set_ylim(-2.5, 4)
    ax.axis("off")

    # Text row
    for idx, ch in enumerate(txt):
        rect = plt.Rectangle((idx - 0.4, 0.5), 0.8, 0.8, fill=False)
        ax.add_patch(rect)
        ax.text(idx, 0.9, ch, ha="center", va="center", fontsize=12)

    status = ax.text(0, 3.5, "", ha="left", va="center", fontsize=9)
    hash_info = ax.text(0, 3.0, "", ha="left", va="center", fontsize=9)
    window_info = ax.text(0, 2.5, "", ha="left", va="center", fontsize=9)

    def update(frame):
        i, w_hash, p_hash, occ, action = frame
        # cleanup previous overlays
        for patch in [p for p in ax.patches if getattr(p, "custom_occ", False)]:
            patch.remove()

        # Highlight current window
        for k in range(m):
            win_rect = plt.Rectangle((i + k - 0.4, 0.5), 0.8, 0.8, fill=True, alpha=0.15)
            win_rect.custom_occ = True
            ax.add_patch(win_rect)

        # Highlight verified matches
        for oc in occ:
            for k in range(m):
                overlay = plt.Rectangle((oc + k - 0.4, 0.5), 0.8, 0.8, fill=True, alpha=0.3, edgecolor="green")
                overlay.custom_occ = True
                ax.add_patch(overlay)

        status.set_text(f"RK: pos={i}, Action={action}, Matches={occ}")
        hash_info.set_text(f"Window Hash={w_hash} | Pattern Hash={p_hash}")
        window_info.set_text(f"Window Text: '{txt[i:i+m]}'")
        return [status, hash_info, window_info]

    anim = FuncAnimation(fig, update, frames=step_data, interval=interval, blit=False, repeat=False)
    plt.close(fig)
    display(HTML(anim.to_jshtml()))
    return occurrences

# ------------------ Utility: Compare All ------------------
def compare_all(pat, txt):
    print("\n=== Comparative Run ===")
    t0 = time.time()
    occ_naive = naive_search(pat, txt)
    dt_naive = time.time() - t0

    t1 = time.time()
    occ_kmp = kmp_search(pat, txt, animate=False)
    dt_kmp = time.time() - t1

    t2 = time.time()
    occ_rk = rabin_karp_search(pat, txt, animate=False)
    dt_rk = time.time() - t2

    print("\n--- Timings (seconds) ---")
    print(f"Naive:      {dt_naive:.6f}")
    print(f"KMP:        {dt_kmp:.6f}")
    print(f"Rabin-Karp: {dt_rk:.6f}")

    # Sanity check: results should agree
    assert occ_kmp == occ_naive == occ_rk, "Mismatch among algorithms!"

# ------------------ Example run ------------------
if __name__ == "__main__":
    text = "ABABDABACDABABCABAB"
    pattern = "ABABCABAB"
    # Comparative textual output
    compare_all(pattern, text)
    # Animated demos (one by one)
    kmp_search(pattern, text, animate=True, interval=400)
    rabin_karp_search(pattern, text, animate=True, interval=400)



=== Comparative Run ===
Naive: Pattern 'ABABCABAB' in text -> [10]
KMP: Pattern 'ABABCABAB' in text -> [10]
Rabin-Karp: Pattern 'ABABCABAB' in text -> [10] (mod=101)

--- Timings (seconds) ---
Naive:      0.000000
KMP:        0.000000
Rabin-Karp: 0.000000
KMP: Pattern 'ABABCABAB' in text -> [10]


Rabin-Karp: Pattern 'ABABCABAB' in text -> [10] (mod=101)
